In [1]:
# ============================================================
# SPSV Project - Phase 3: Feature Engineering & Dataset Integration


# In step 1, I cleaned the five datasets.
# In step 2, I performed EDA and found issues such as inconsistent categories (e.g., YES/NO vs Y/N).
#
# In stage 3, I am preparing the datasets for:
# 1) Dashboard KPIs (counts, rates, trends by month/county/licence type)
# 2) Machine learning (a single "ML-ready" table with engineered features)
#
# This stage focuses on:
# - Creating meaningful features (binary flags, durations, rates, time buckets)
# - Integrating datasets (linking complaints/inspections/enforcement back to licences)
# - Producing summary tables for the dashboard and an ML dataset
# ============================================================

import pandas as pd
import numpy as np

In [3]:
# ------------------------------------------------------------
# 1) LOAD CLEANED DATA
# ------------------------------------------------------------

# I load cleaned data to ensure I am building features on consistent categories and parsed dates.

def load_csv(preferred_path: str, fallback_path: str) -> pd.DataFrame:
    try:
        return pd.read_csv(preferred_path)
    except FileNotFoundError:
        return pd.read_csv(fallback_path)

complaints = load_csv("spsv_complaints_clean.csv", "spsv_complaints.csv")
enforcement = load_csv("spsv_enforcement_clean.csv", "spsv_enforcement.csv")
inspections = load_csv("spsv_inspections_clean.csv", "spsv_inspections.csv")
licences = load_csv("spsv_licences_clean.csv", "spsv_licences.csv")
monthly_kpis = load_csv("spsv_monthly_kpis_clean.csv", "spsv_monthly_kpis.csv")

# ------------------------------------------------------------
# 2) RE-CONFIRM DATETIME COLUMNS (required for time features)
# ------------------------------------------------------------

# Time-based features (monthly trends, rolling averages, expiry windows) require datetime columns.
def to_datetime_safe(df: pd.DataFrame, col: str) -> None:
    if col in df.columns:
        df[col] = pd.to_datetime(df[col], errors="coerce")

# These column names are based on what we used in Phase 2.
# If your actual columns differ, the code will simply skip missing columns.
to_datetime_safe(complaints, "Date_Received")
to_datetime_safe(enforcement, "Action_Date")
to_datetime_safe(inspections, "Inspection_Date")
to_datetime_safe(licences, "Issue_Date")
to_datetime_safe(licences, "Expiry_Date")
to_datetime_safe(monthly_kpis, "Month")


In [4]:
# ------------------------------------------------------------
# 3) QUICK CHECK OF COLUMN NAMES
# ------------------------------------------------------------
print("\n--- Column names (quick check) ---")
print("licences:", list(licences.columns))
print("complaints:", list(complaints.columns))
print("inspections:", list(inspections.columns))
print("enforcement:", list(enforcement.columns))
print("monthly_kpis:", list(monthly_kpis.columns))


--- Column names (quick check) ---
licences: ['Licence_ID', 'Licence_Type', 'Issue_Date', 'Expiry_Date', 'Status', 'County', 'Wheelchair_Accessible', 'Vehicle_Age', 'Vehicle_Plate_Year', 'Driver_Experience_Years', 'Fleet_Segment']
complaints: ['Complaint_ID', 'Licence_ID', 'Complaint_Type', 'Date_Received', 'Resolved', 'Days_To_Resolution', 'Escalated']
inspections: ['Inspection_ID', 'Licence_ID', 'Inspection_Date', 'Inspection_Type', 'Outcome', 'Breach_Category', 'Follow_Up_Required']
enforcement: ['Enforcement_ID', 'Licence_ID', 'Action_Type', 'Action_Date', 'Outcome']
monthly_kpis: ['Month', 'County', 'Inspections', 'Failures', 'Follow_Ups', 'Complaints', 'Escalations', 'Failure_Rate']


In [5]:
# ------------------------------------------------------------
# 4) FIX A KNOWN DATA QUALITY ISSUE FOUND IN EDA (Wheelchair Accessible)
# ------------------------------------------------------------

# EDA revealed inconsistent encoding in Wheelchair_Accessible (YES/NO vs Y/N).
# I standardise it here because it directly affects accessibility KPIs and dashboard accuracy.
if "Wheelchair_Accessible" in licences.columns:
    licences["Wheelchair_Accessible"] = (
        licences["Wheelchair_Accessible"]
        .replace({"Y": "YES", "N": "NO"})
        .astype(str)
        .str.upper()
    )

# ------------------------------------------------------------
# 5) FEATURE ENGINEERING: LICENCES (licence-level features)
# ------------------------------------------------------------

# The licences dataset is the "anchor" table because it contains geography (County),
# licence type, and other stable attributes. Other datasets will be linked to it.
lic = licences.copy()

# 5.1 Binary accessibility flag
if "Wheelchair_Accessible" in lic.columns:
    lic["Is_Wheelchair_Accessible"] = (lic["Wheelchair_Accessible"] == "YES").astype(int)

# 5.2 Vehicle age band (makes it more dashboard-friendly)
if "Vehicle_Age" in lic.columns:
    lic["Vehicle_Age_Band"] = pd.cut(
        lic["Vehicle_Age"],
        bins=[-np.inf, 2, 5, 10, np.inf],
        labels=["0-2", "3-5", "6-10", "11+"]
    )

# 5.3 Licence duration + expiring soon flag

# Duration and expiry windows are useful for forecasting renewals and identifying future workload.
if "Issue_Date" in lic.columns and "Expiry_Date" in lic.columns:
    lic["Licence_Duration_Days"] = (lic["Expiry_Date"] - lic["Issue_Date"]).dt.days

    # "Expiring soon" is defined as expiring within the next 90 days from the latest date in the data.
    reference_date = lic["Expiry_Date"].max()
    lic["Days_To_Expiry"] = (lic["Expiry_Date"] - reference_date).dt.days
    lic["Is_Expiring_90_Days"] = ((lic["Days_To_Expiry"] <= 90) & (lic["Days_To_Expiry"] >= 0)).astype(int)

# 5.4 Standardise main categorical columns (small safety step)
for col in ["County", "Licence_Type", "Status"]:
    if col in lic.columns:
        lic[col] = lic[col].astype(str).str.upper().str.strip()

# ------------------------------------------------------------
# 6) FEATURE ENGINEERING: COMPLAINTS (complaint-level features)
# ------------------------------------------------------------

# Complaints are a potential "risk signal". Here I create useful flags and time fields.
comp = complaints.copy()

# 6.1 Month field for grouping
if "Date_Received" in comp.columns:
    comp["Month"] = comp["Date_Received"].dt.to_period("M").dt.to_timestamp()

# 6.2 Binary flags for escalated/resolved (works even if values are YES/NO, Y/N, True/False)
def to_binary_yes_no(series: pd.Series) -> pd.Series:
    s = series.astype(str).str.upper().str.strip()
    return s.isin(["YES", "Y", "TRUE", "1"]).astype(int)

if "Escalated" in comp.columns:
    comp["Is_Escalated"] = to_binary_yes_no(comp["Escalated"])

if "Resolved" in comp.columns:
    comp["Is_Resolved"] = to_binary_yes_no(comp["Resolved"])

# 6.3 Resolution speed band (dashboard-friendly)
if "Days_To_Resolution" in comp.columns:
    comp["Resolution_Speed_Band"] = pd.cut(
        comp["Days_To_Resolution"],
        bins=[-np.inf, 7, 14, 30, np.inf],
        labels=["<=7 days", "8-14 days", "15-30 days", "31+ days"]
    )

# 6.4 Standardise complaint type for grouping
if "Complaint_Type" in comp.columns:
    comp["Complaint_Type"] = comp["Complaint_Type"].astype(str).str.upper().str.strip()

# ------------------------------------------------------------
# 7) FEATURE ENGINEERING: INSPECTIONS (inspection-level features)
# ------------------------------------------------------------

# Inspections are core compliance monitoring signals. I create a clear "Is_Fail" flag
# to support county risk scoring and monthly failure rates.
insp = inspections.copy()

if "Inspection_Date" in insp.columns:
    insp["Month"] = insp["Inspection_Date"].dt.to_period("M").dt.to_timestamp()

# "Fail" rules (simple and transparent)
failure_labels = {"FAIL", "FAILED", "NON-COMPLIANT", "NON COMPLIANT"}
if "Outcome" in insp.columns:
    insp["Is_Fail"] = insp["Outcome"].astype(str).str.upper().isin(failure_labels).astype(int)

# Follow-up required flag
if "Follow_Up_Required" in insp.columns:
    # handles YES/NO, Y/N, etc
    insp["Is_Follow_Up_Required"] = to_binary_yes_no(insp["Follow_Up_Required"])

# Breach category standardisation
if "Breach_Category" in insp.columns:
    insp["Breach_Category"] = insp["Breach_Category"].astype(str).str.upper().str.strip()

# ------------------------------------------------------------
# 8) FEATURE ENGINEERING: ENFORCEMENT (action-level features)
# ------------------------------------------------------------

# Enforcement actions represent regulatory response. I create fields that help summarise severity and frequency.
enf = enforcement.copy()

if "Action_Date" in enf.columns:
    enf["Month"] = enf["Action_Date"].dt.to_period("M").dt.to_timestamp()

for col in ["Action_Type", "Outcome"]:
    if col in enf.columns:
        enf[col] = enf[col].astype(str).str.upper().str.strip()

# ------------------------------------------------------------
# 9) DATASET INTEGRATION (link everything back to licences)
# ------------------------------------------------------------

# I join complaints/inspections/enforcement to licences using Licence_ID.
# This enables analysis by County, Licence Type and Status.

# Safety: only attempt merges if Licence_ID exists in both tables
def safe_merge(left: pd.DataFrame, right: pd.DataFrame, on: str, how: str = "left") -> pd.DataFrame:
    if on in left.columns and on in right.columns:
        return left.merge(right, on=on, how=how)
    else:
        print(f"WARNING: Could not merge because '{on}' not found in both tables.")
        return left.copy()

# Licence columns to bring into operational tables
lic_key_cols = [c for c in ["Licence_ID", "County", "Licence_Type", "Status", "Is_Wheelchair_Accessible", "Vehicle_Age", "Vehicle_Age_Band"] if c in lic.columns]

comp_joined = safe_merge(comp, lic[lic_key_cols], on="Licence_ID", how="left")
insp_joined = safe_merge(insp, lic[lic_key_cols], on="Licence_ID", how="left")
enf_joined  = safe_merge(enf,  lic[lic_key_cols], on="Licence_ID", how="left")

# ------------------------------------------------------------
# 10) DASHBOARD-READY SUMMARY TABLES
# ------------------------------------------------------------

# Dashboards work best with aggregated tables (County/month summaries, KPI metrics).
# I create two main summary outputs:
# 1) county_monthly_summary: risk indicators by county and month
# 2) licence_level_features: one row per licence with counts/flags (useful for ML)

# 10.1 County-month summary from operational datasets
# Complaints: counts + escalation rate
if "Month" in comp_joined.columns and "County" in comp_joined.columns:
    county_month_complaints = (
        comp_joined
        .groupby(["County", "Month"])
        .agg(
            Complaints=("Complaint_ID", "count") if "Complaint_ID" in comp_joined.columns else ("Licence_ID", "count"),
            Escalations=("Is_Escalated", "sum") if "Is_Escalated" in comp_joined.columns else ("Licence_ID", "count")
        )
        .reset_index()
    )
    if "Escalations" in county_month_complaints.columns and "Complaints" in county_month_complaints.columns:
        county_month_complaints["Escalation_Rate"] = county_month_complaints["Escalations"] / county_month_complaints["Complaints"]
else:
    county_month_complaints = pd.DataFrame()

# Inspections: counts + failures + failure rate + follow-ups
if "Month" in insp_joined.columns and "County" in insp_joined.columns:
    county_month_inspections = (
        insp_joined
        .groupby(["County", "Month"])
        .agg(
            Inspections=("Inspection_ID", "count") if "Inspection_ID" in insp_joined.columns else ("Licence_ID", "count"),
            Failures=("Is_Fail", "sum") if "Is_Fail" in insp_joined.columns else ("Licence_ID", "count"),
            Follow_Ups=("Is_Follow_Up_Required", "sum") if "Is_Follow_Up_Required" in insp_joined.columns else ("Licence_ID", "count"),
        )
        .reset_index()
    )
    county_month_inspections["Failure_Rate"] = county_month_inspections["Failures"] / county_month_inspections["Inspections"]
else:
    county_month_inspections = pd.DataFrame()

# Enforcement: counts by county-month
if "Month" in enf_joined.columns and "County" in enf_joined.columns:
    county_month_enforcement = (
        enf_joined
        .groupby(["County", "Month"])
        .agg(
            Enforcement_Actions=("Enforcement_ID", "count") if "Enforcement_ID" in enf_joined.columns else ("Licence_ID", "count")
        )
        .reset_index()
    )
else:
    county_month_enforcement = pd.DataFrame()

# Combine county-month summaries into one table

# I do an outer join to avoid losing county-month rows that exist in one dataset but not another.
county_monthly_summary = None

# Start with inspections (often the main compliance timeline)
if not county_month_inspections.empty:
    county_monthly_summary = county_month_inspections.copy()
else:
    # If inspections summary isn't available, start with complaints summary
    county_monthly_summary = county_month_complaints.copy() if not county_month_complaints.empty else pd.DataFrame()

if not county_month_complaints.empty:
    county_monthly_summary = county_monthly_summary.merge(
        county_month_complaints, on=["County", "Month"], how="outer"
    )

if not county_month_enforcement.empty:
    county_monthly_summary = county_monthly_summary.merge(
        county_month_enforcement, on=["County", "Month"], how="outer"
    )

# Fill remaining NaNs in numeric summary columns with 0 (because missing here often means "no events that month")
num_cols = county_monthly_summary.select_dtypes(include=[np.number]).columns
county_monthly_summary[num_cols] = county_monthly_summary[num_cols].fillna(0)

# 10.2 Licence-level features (one row per licence)

# This is useful for ML (e.g., predict high-risk licences) and also for dashboard drill-down.
# I aggregate events per Licence_ID and attach them back to the licence table.

# Complaints per licence
if "Licence_ID" in comp.columns:
    comp_per_lic = (
        comp.groupby("Licence_ID")
            .agg(
                Complaints=("Complaint_ID", "count") if "Complaint_ID" in comp.columns else ("Licence_ID", "count"),
                Escalations=("Is_Escalated", "sum") if "Is_Escalated" in comp.columns else ("Licence_ID", "count"),
                Avg_Days_To_Resolution=("Days_To_Resolution", "mean") if "Days_To_Resolution" in comp.columns else ("Licence_ID", "count")
            )
            .reset_index()
    )
else:
    comp_per_lic = pd.DataFrame()

# Inspections per licence
if "Licence_ID" in insp.columns:
    insp_per_lic = (
        insp.groupby("Licence_ID")
            .agg(
                Inspections=("Inspection_ID", "count") if "Inspection_ID" in insp.columns else ("Licence_ID", "count"),
                Failures=("Is_Fail", "sum") if "Is_Fail" in insp.columns else ("Licence_ID", "count"),
                Follow_Ups=("Is_Follow_Up_Required", "sum") if "Is_Follow_Up_Required" in insp.columns else ("Licence_ID", "count"),
            )
            .reset_index()
    )
    insp_per_lic["Failure_Rate"] = insp_per_lic["Failures"] / insp_per_lic["Inspections"]
else:
    insp_per_lic = pd.DataFrame()

# Enforcement per licence
if "Licence_ID" in enf.columns:
    enf_per_lic = (
        enf.groupby("Licence_ID")
           .agg(
                Enforcement_Actions=("Enforcement_ID", "count") if "Enforcement_ID" in enf.columns else ("Licence_ID", "count")
           )
           .reset_index()
    )
else:
    enf_per_lic = pd.DataFrame()

# Start with licence table and merge in aggregated event metrics
licence_level_features = lic.copy()

if not comp_per_lic.empty:
    licence_level_features = licence_level_features.merge(comp_per_lic, on="Licence_ID", how="left")
if not insp_per_lic.empty:
    licence_level_features = licence_level_features.merge(insp_per_lic, on="Licence_ID", how="left")
if not enf_per_lic.empty:
    licence_level_features = licence_level_features.merge(enf_per_lic, on="Licence_ID", how="left")

# Replace NaNs created by merges with 0 for event counts (means "no events recorded")
for col in ["Complaints", "Escalations", "Inspections", "Failures", "Follow_Ups", "Enforcement_Actions"]:
    if col in licence_level_features.columns:
        licence_level_features[col] = licence_level_features[col].fillna(0)

# ------------------------------------------------------------
# 11) ML-READY DATASET (simple + transparent)
# ------------------------------------------------------------

# This is a basic ML-ready dataset. It is NOT the final model yet.
# The aim is to create a table with numeric/categorical features that can be encoded later.

ml_ready = licence_level_features.copy()

# A simple example target (can be refined):
# "High_Risk" = 1 if the licence has >=1 inspection failure OR >=2 complaints OR >=1 enforcement action.

# This is an interpretable rule-based target suitable for synthetic data, and can be adjusted.
high_risk_conditions = []
if "Failures" in ml_ready.columns:
    high_risk_conditions.append(ml_ready["Failures"] >= 1)
if "Complaints" in ml_ready.columns:
    high_risk_conditions.append(ml_ready["Complaints"] >= 2)
if "Enforcement_Actions" in ml_ready.columns:
    high_risk_conditions.append(ml_ready["Enforcement_Actions"] >= 1)

if high_risk_conditions:
    ml_ready["High_Risk"] = np.logical_or.reduce(high_risk_conditions).astype(int)
else:
    ml_ready["High_Risk"] = 0  # fallback if event columns are missing


In [6]:
# ------------------------------------------------------------
# 12) SAVE OUTPUTS
# ------------------------------------------------------------

# Saving these outputs supports reproducibility and makes Phase 4 and Phase 5 easier.
county_monthly_summary.to_csv("county_monthly_summary.csv", index=False)
licence_level_features.to_csv("licence_level_features.csv", index=False)
ml_ready.to_csv("ml_ready_dataset.csv", index=False)

print("\nPhase 3 complete. Saved files:")
print("- county_monthly_summary.csv (dashboard aggregated table)")
print("- licence_level_features.csv (dashboard drill-down + ML features)")
print("- ml_ready_dataset.csv (ML dataset with a simple High_Risk target)")


Phase 3 complete. Saved files:
- county_monthly_summary.csv (dashboard aggregated table)
- licence_level_features.csv (dashboard drill-down + ML features)
- ml_ready_dataset.csv (ML dataset with a simple High_Risk target)


In [7]:
# ------------------------------------------------------------
# 13) QUICK PREVIEW 
# ------------------------------------------------------------
print("\n--- Preview: county_monthly_summary ---")
print(county_monthly_summary.head())

print("\n--- Preview: licence_level_features ---")
print(licence_level_features.head())

print("\n--- Preview: ml_ready_dataset (target distribution) ---")
print(ml_ready["High_Risk"].value_counts())


--- Preview: county_monthly_summary ---
  County      Month  Inspections  Failures  Follow_Ups  Failure_Rate  \
0  CAVAN 2022-01-01           14         2           1      0.142857   
1  CAVAN 2022-02-01           14         2           3      0.142857   
2  CAVAN 2022-03-01           15         4           3      0.266667   
3  CAVAN 2022-04-01           14         3           2      0.214286   
4  CAVAN 2022-05-01           25         4           2      0.160000   

   Complaints  Escalations  Escalation_Rate  Enforcement_Actions  
0        13.0          2.0         0.153846                  6.0  
1        13.0          2.0         0.153846                  1.0  
2        14.0          3.0         0.214286                  3.0  
3        18.0          2.0         0.111111                  5.0  
4        11.0          3.0         0.272727                  4.0  

--- Preview: licence_level_features ---
   Licence_ID Licence_Type Issue_Date Expiry_Date    Status     County  \
0  SPSV_0